In [1]:
%install-location $cwd/swift-install
%install '.package(url: "https://github.com/mxcl/Path.swift", from: "0.16.1")' Path
%install '.package(url: "https://github.com/saeta/Just", from: "0.7.2")' Just
%install '.package(url: "https://github.com/latenitesoft/NotebookExport", from: "0.5.0")' NotebookExport

Installing packages:
	.package(url: "https://github.com/mxcl/Path.swift", from: "0.16.1")
		Path
	.package(url: "https://github.com/saeta/Just", from: "0.7.2")
		Just
	.package(url: "https://github.com/latenitesoft/NotebookExport", from: "0.5.0")
		NotebookExport
With SwiftPM flags: []
Working in: /tmp/tmp6or8tkxt/swift-install
[1/3] Compiling jupyterInstalledPackages jupyterInstalledPackages.swift
[2/4] Merging module jupyterInstalledPackages
clang-7: error: unable to execute command: Bus error
clang-7: error: linker command failed due to signal (use -v to see invocation)
<unknown>:0: error: link command failed with exit code 254 (use -v to see invocation)
[2/3] Linking libjupyterInstalledPackages.so


: 

In [ ]:
// export

// Source: The original python implementation can be found here:
// https://github.com/mortendahl/privateml/blob/master/image-analysis/pond/tensor.py

import TensorFlow

let Q:Int64 = 12332434434

let BASE = 3
let PRECISION_INTEGRAL   = 6
let PRECISION_FRACTIONAL = 5
let SCALING_FACTOR = pow(Double(BASE), Double(PRECISION_FRACTIONAL))


func mod(_ x: Tensor<Int64>, _ q: Int64) -> Tensor<Int64>{
  return Raw.floorMod(x, Tensor<Int64>([q]))
}

extension TensorShape {
  func countElements() -> Int{
    var nb_el = 1
    for i in 0 ..< self.count{
      nb_el = nb_el * self[i]
    }
    return nb_el
  }
}

public func _encode(rationals: Tensor<Double>) -> Tensor<Int64>{
    let scaled = rationals * SCALING_FACTOR
    let integer = mod(Tensor<Int64>(scaled), Q)
    return integer
}

func _decode(field_element: Tensor<Int64>) -> Tensor<Double>{
      let gr_q = Tensor<Int64>(_Raw.greaterEqual(field_element, Tensor<Int64>([Q/2]))) // 1 if val > Q/2 else 0
      let scaled = field_element - gr_q * Q
      let scaled_rationals = Tensor<Double>(scaled)
      return scaled_rationals / SCALING_FACTOR
}

// NOTE Should be able to use Tensor(randomUniform instead)
func makeRandomList(_ n: Int, _ q: Int64) -> [Int64] {
    return (0..<n).map { _ in Int64.random(in: 0..<q) }
}

func _share(x: Tensor<Int64>) ->  (Tensor<Int64>, Tensor<Int64>){
    let share0 = Tensor<Int64>(makeRandomList(x.shape.countElements(), Q)).reshaped(to: x.shape)
    let share1 = mod(x - share0, Q)
    return (share0, share1)
}

func reconstruct(share0: Tensor<Int64>, share1: Tensor<Int64>) -> Tensor<Int64>{
  let rec = mod(share0 + share1, Q)
  return rec
}

In [ ]:
let x: Tensor<Double> = Tensor([[1.0, 2.0], [3.0, 4.0]])
let x_enc = _encode(rationals: x)
let x_shared = _share(x: x_enc)
let x_rec = _decode(field_element: reconstruct(share0: x_shared.0, share1: x_shared.1))
print(x_rec)

In [ ]:
// export
enum TensorError: Error{
  case tensorType(String)
}

func errorTypeMessage<T, U>(_ x: T, _ y: U) -> String {
  return "\(type(of: x)) doesn't support  \(type(of: y))"
}

public protocol PondTensor {}

// PublicTensor

public struct PublicTensor: PondTensor {
  let elements: Tensor<Int64>

  init(elements: Tensor<Int64>){
    self.elements = elements
  }

  public static func from_values(values: Tensor<Double>) -> PublicTensor{
    let enc_val = _encode(rationals: values)

    return PublicTensor(elements: enc_val)
  }

  public func unwrap() -> Tensor<Int64>{
    return self.elements
  }

  public func reveal() -> PublicTensor{
    return self
  }

  public func decode() -> Tensor<Double>{
    return _decode(field_element: self.elements)
  }
    
  public func truncate() -> PublicTensor {
    let positive_numbers = Tensor<Int64>(_Raw.lessEqual(self.elements, Tensor<Int64>([Q/2])))
    var elements = self.elements
    elements = mod((Q + (2 * positive_numbers - 1) * elements), Q)
    elements = _Raw.floorDiv(elements, Tensor<Int64>([Int64(SCALING_FACTOR)]))
    elements = mod((Q + (2 * positive_numbers - 1) * elements), Q) //#

    return PublicTensor(elements: elements)
   }
 }


func add(x: PublicTensor, y: PublicTensor) -> PublicTensor{
    let elements = mod(x.elements + y.elements, Q)
    return PublicTensor(elements: elements)
}
    
func mul(x: PublicTensor, y: PublicTensor) -> PublicTensor{
    let elements = mod(_Raw.mul(x.elements, y.elements), Q)
    return PublicTensor(elements: elements).truncate()
}


public func matmul(x: PublicTensor, y: PublicTensor) -> PublicTensor{
    let elements = mod(_Raw.matMul(x.elements, y.elements), Q)
    return PublicTensor(elements: elements).truncate()
}


// PublicFieldTensor

public struct PublicFieldTensor: PondTensor {
  let elements: Tensor<Int64>

  init(elements: Tensor<Int64>){
    self.elements = elements
  }

  public static func from_values(values: Tensor<Int64>) -> PublicFieldTensor{
    let enc_val = values
    return PublicFieldTensor(elements: enc_val)
  }

  public func reveal() -> PublicFieldTensor {
    return self
  }
}

public func add(x: PublicFieldTensor, y: PublicFieldTensor) -> PublicFieldTensor{
    let elements = mod(x.elements + y.elements, Q)
    return PublicFieldTensor(elements: elements)
  }

public func add(x: PublicFieldTensor, y: PrivateFieldTensor) -> PrivateFieldTensor{
    let share0 = mod(x.elements + y.share0, Q)
    let share1 = y.share1
    return PrivateFieldTensor(share0: share0, share1: share1)
}

func matmul(x: PublicFieldTensor, y: PublicFieldTensor) -> PublicFieldTensor{
    let elements = Raw.matMul(x.elements, y.elements) % Q
    return PublicFieldTensor(elements: elements)
}

func matmul(x: PublicFieldTensor, y: PrivateFieldTensor) -> PrivateFieldTensor{
    let share0 = mod(Raw.matMul(x.elements, y.share0), Q)
    let share1 = mod(Raw.matMul(x.elements, y.share1), Q)
    return PrivateFieldTensor(share0: share0, share1: share1)
}


// PrivateFielTensor 

public struct PrivateFieldTensor: PondTensor{
  let share0: Tensor<Int64>
  let share1: Tensor<Int64>

  init(share0: Tensor<Int64>, share1: Tensor<Int64>){
    self.share0 = share0
    self.share1 = share1
  }

  public static func from_values(values: Tensor<Int64>) -> PrivateFieldTensor{
    let x_shared = _share(x: values)
    return PrivateFieldTensor(share0: x_shared.0, share1: x_shared.1)
  }

  public func reveal() -> PublicFieldTensor{
  let x_rec = reconstruct(share0: self.share0, share1: self.share1)
    return PublicFieldTensor.from_values(values: x_rec)
  }
}

public func add(x: PrivateFieldTensor, y: PrivateFieldTensor) -> PrivateFieldTensor{
    let share0 = mod(x.share0 + y.share0, Q)
    let share1 = mod(x.share1 + y.share1, Q)
    return PrivateFieldTensor(share0: share0, share1: share1)
}
    
 public func add(x: PrivateFieldTensor, y: PublicFieldTensor) -> PrivateFieldTensor{
    let share0 = mod(x.share0 + y.elements, Q)
    let share1 = mod(x.share1, Q)
    return PrivateFieldTensor(share0: share0, share1: share1)
  }

public func matmul(x: PrivateFieldTensor, y: PublicFieldTensor) -> PrivateFieldTensor{
    let share0 = Raw.matMul(x.share0, y.elements) % Q
    let share1 = Raw.matMul(x.share1, y.elements) % Q
    return PrivateFieldTensor(share0: share0, share1: share1)
    }

In [ ]:
let x_p = PublicTensor.from_values(values: Tensor<Double>([1, 2, 3, 4]))
let y_p = PublicTensor.from_values(values: Tensor<Double>([1, 2, 3, 4]))
let out_p = add(x:x_p , y:y_p)
print(out_p.reveal().decode())

In [ ]:
// export
func generate_dot_triple(m: Int, n: Int, o: Int) -> (PrivateFieldTensor, 
                                                     PrivateFieldTensor, 
                                                     PrivateFieldTensor){

    let a = Tensor<Int64>(makeRandomList(m*n, Q)).reshaped(to: [m, n])
    let b = Tensor<Int64>(makeRandomList(n*o, Q)).reshaped(to: [n, o])
    let ab = Raw.matMul(a, b)
    return (PrivateFieldTensor.from_values(values: Tensor<Int64>(a)), 
            PrivateFieldTensor.from_values(values: Tensor<Int64>(b)), 
            PrivateFieldTensor.from_values(values: Tensor<Int64>(ab)))
}

In [ ]:
// export
public struct PrivateTensor: PondTensor {
  let share0: Tensor<Int64>
  let share1: Tensor<Int64>

  var shape: TensorShape {
        return share0.shape
    }

  init(share0: Tensor<Int64>, share1: Tensor<Int64>){
    self.share0 = share0
    self.share1 = share1
  }

  public static func from_values(values: Tensor<Double>) -> PrivateTensor{
    let x_shared = _share(x: _encode(rationals: values))
    return PrivateTensor(share0: x_shared.0, share1: x_shared.1)
  }

  public func reveal() -> PublicTensor{
    let x_rec = _decode(field_element: reconstruct(share0: self.share0, share1: self.share1))
    return PublicTensor.from_values(values: x_rec)
  }

  public func truncate() -> PrivateTensor {
    let share0 = _Raw.floorDiv(self.share0, Tensor<Int64>([Int64(SCALING_FACTOR)]))
    let share1 =  (Q - _Raw.floorDiv(Q - self.share1, Tensor<Int64>([Int64(SCALING_FACTOR)]))) % Q
    return PrivateTensor(share0: share0, share1: share1)
   }
}

func add(x: PrivateTensor, y: PublicTensor) -> PrivateTensor{
    let share0 =  mod(x.share0 + y.elements, Q)
    let share1 = x.share1
    return PrivateTensor(share0: share0, share1: share1)
  }
    
func add(x: PrivateTensor, y: PrivateTensor) -> PrivateTensor{
    let share0 = mod(x.share0 + y.share0, Q)
    let share1 = mod(x.share1 + y.share1, Q)
    return PrivateTensor(share0: share0, share1: share1)
}

func sub(x: PrivateTensor, y: PublicTensor) -> PrivateTensor{
    let share0 = mod(x.share0 - y.elements, Q)
    let share1 = x.share1
    return PrivateTensor(share0: share0, share1: share1)
}

func sub(x: PrivateTensor, y: PrivateTensor) -> PrivateTensor{
    let share0 = mod(x.share0 - y.share0, Q)
    let share1 = mod(x.share1 - y.share1, Q)
    return PrivateTensor(share0: share0, share1: share1)
}

func sub(x: PrivateTensor, y: PrivateFieldTensor) -> PrivateFieldTensor{
  let share0 = mod(x.share0 - y.share0, Q)
  let share1 = mod(x.share1 - y.share1, Q)
  return PrivateFieldTensor(share0: share0, share1: share1)
}


public func matmul(x: PrivateTensor, y: PrivateTensor) -> PrivateTensor{
    let m = x.shape[0]
    let n = x.shape[1]
    let o = y.shape[1]

    let triple = generate_dot_triple(m: m, n: n, o: o)
    let a = triple.0 
    let b = triple.1 
    let ab = triple.2

    let alpha_r = sub(x: x, y: a).reveal()
    let beta_r = sub(x: y, y: b).reveal()
    let z_1 = matmul(x: alpha_r, y: beta_r)
    let z_2 = matmul(x: alpha_r, y: b)
    let z_3 = matmul(x: a, y: beta_r)
    let z = add(x: add(x: add(x: z_1, y: z_2), y: z_3), y: ab)

    return PrivateTensor(share0: z.share0 , share1: z.share1).truncate()
}

In [ ]:
let x = PrivateTensor.from_values(values: Tensor([[1, 2], [3, 4]]))
let b = PrivateTensor.from_values(values: Tensor([[1, 2], [3, 4]]))
let c = PrivateFieldTensor.from_values(values: Tensor([[1, 2], [3, 4]]))
let res_add = add(x: x, y: b)
let res_sub = sub(x: x, y: b)


print(res_add.reveal().decode())
print(res_sub.reveal().decode())

In [ ]:
import NotebookExport
import Path
let exporter = NotebookExport(Path.cwd/"tensor.ipynb")
print(exporter.export(usingPrefix: "ppmlNB_"))